In [ ]:
#this is multinomial naive bayes
#load data into dataframe
#divide the data and the target columns
#apply count vectorizer on data columns ie lyrics
#count vectorizer will tokenize, stememetize and remove stopwords,give a unique no to each word and create a vector for each document showing whether that word is present in that document or not
#apply tfidf on the result obtaind from count vectorizer
#tfidf gives the product of term frequency and inverse document frequency
#term frequency is the no of times that word occurs in that documnet
#inverse document frequency is the no of documents in which it appears
#tfidf gives higher weightage to those words that appears in less no of documents 
#select multinomialNB as the model 
#to the same with the test lyrics and predict the moods using the model proposed

In [1]:
#loads the csv file into data frame
import pandas as pd

df = pd.read_csv('/home/pooja/projectACMS/train_lyrics_1511.csv')

df.head()

,artist,title,lyrics,mood,year
0,Casual,I Didn't Mean To,Verse One:\n\nAlright I might\nHave had a litt...,sad,1994.0
1,Adam Ant,Something Girls,Adam Ant/Marco Pirroni\nEvery girl is a someth...,happy,1982.0
2,Gob,Face the Ashes,"I've just erased it's been a while, I've got a...",sad,2007.0
3,Lionel Richie,Tonight Will Be Alright,Little darling \nWhere you've been so long \nI...,happy,1986.0
4,Blue Rodeo,Floating,"Lead Vocal by Greg\n\nWell, these late night c...",sad,1987.0


In [2]:
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np

X_train = df['lyrics'].values 

y_train = df['mood'].values

print('before: %s ...' %y_train[:5])

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

print('after: %s ...' %y_train[:5])

before: ['sad' 'happy' 'sad' 'happy' 'sad'] ...
after: [1 0 1 0 1] ...


In [3]:
print(y_train[:10])

[1 0 1 0 1 0 0 0 1 1]


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(1511, 13168)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(1511, 13168)

In [6]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1511, 13168)

In [7]:
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB().fit(X_train_tfidf, y_train)

In [8]:
import pandas as pd

df1 = pd.read_csv('/home/pooja/projectACMS/valid_lyrics_200.csv')

df1.head()

,artist,title,lyrics,mood
0,Placebo,Where Is My Mind (XFM Live Version),Ooooohh\nOoooohh\nOoooohh\nOoooohh\n\nWith you...,happy
1,Queens Of The Stone Age,This Lullaby,"Where, oh, where have you been, my love?\nWher...",sad
2,MC5,Looking At You (Cody High School),When it happened \nsomething snapped inside \n...,happy
3,Dimmu Borgir,The Fundamental Alienation,My eyes got blinded\nAnd conned by the light\n...,sad
4,Dark Fortress,The Silver Gate,"[Music: V Santura, Lyrics: Morean]\n\nAwake, a...",sad


In [9]:
count=0
X_new_counts=[]
X_new_tdidf=[]
X_new_counts = count_vect.transform(df1['lyrics'].values)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf1.predict(X_new_tfidf)
#print(predicted)
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count=count+1
print(count)    

106


In [10]:
print("acc of Naive Bayes =",count/200)

acc of Naive Bayes = 0.53


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
text_clf.fit(X_train, y_train)  
predicted = text_clf.predict(df1['lyrics'].values)
count1=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count1=count1+1
print(count1)


135


In [13]:
print("acc of Sochastic gradient decent =",count1/200)

acc of Sochastic gradient decent = 0.675


In [14]:
from sklearn.svm import SVC
text_clf1=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SVC()),])
text_clf1.fit(X_train, y_train)  
predicted = text_clf1.predict(df1['lyrics'].values)
count2=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count2=count2+1
print(count2)


95


In [15]:
print("acc of Support Vector Machine =", count2/200)

acc of Support Vector Machine = 0.475


In [37]:
from sklearn.ensemble import RandomForestClassifier
text_clf2=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', RandomForestClassifier() ),])
text_clf2.fit(X_train, y_train)  
predicted = text_clf2.predict(df1['lyrics'].values)
count3=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count3=count3+1
print(count3)

117


In [27]:
print("acc of Random Forest =", count3/200)

acc of Random Forest = 0.635


In [28]:
from sklearn.neighbors import KNeighborsClassifier
text_clf3=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', KNeighborsClassifier(n_neighbors=10) ),])
text_clf3.fit(X_train, y_train)  
predicted = text_clf3.predict(df1['lyrics'].values)
count4=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count4=count4+1
print(count4)

125


In [29]:
print("acc of KNN =", count4/200)

acc of KNN = 0.625


In [30]:
from sklearn.linear_model import RidgeClassifier
text_clf4=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf',RidgeClassifier(tol=1e-2, solver="lsqr") ),])
text_clf4.fit(X_train, y_train)  
predicted = text_clf4.predict(df1['lyrics'].values)
count5=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count5=count5+1
print(count5)


136


/home/pooja/.local/lib/python3.5/site-packages/sklearn/linear_model/ridge.py:319: UserWarning: In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.
  warnings.warn("In Ridge, only 'sag' solver can currently fit the "


In [31]:
print("acc of Ridge classifier =", count5/200)

acc of Ridge classifier = 0.68


In [32]:
from sklearn.linear_model import Perceptron
text_clf5=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf',Perceptron(n_iter=50) ),])
text_clf5.fit(X_train, y_train)  
predicted = text_clf5.predict(df1['lyrics'].values)
count6=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count6=count6+1
print(count6)

136


/home/pooja/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [33]:
print("acc of Perceptron =",count6/200)

acc of Perceptron = 0.68


In [34]:
from sklearn.linear_model import PassiveAggressiveClassifier
text_clf6=Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf',PassiveAggressiveClassifier(n_iter=50) ),])
text_clf6.fit(X_train, y_train)  
predicted = text_clf6.predict(df1['lyrics'].values)
count7=0
for row_id in df1.index:
    mood=df1.loc[row_id]['mood']
    if mood=='happy':
        mood1=0
    else:
        mood1=1
    if (predicted[row_id])==mood1:
        count7=count7+1
print(count7)

132


/home/pooja/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [35]:
print("acc of Passive Aggressive Classifier =", count7/200)

acc of Passive Aggressive Classifier = 0.66
